# Sprint13ディープラーニングフレームワーク1
## Sprintの目的
フレームワークのコードを読めるようにする
フレームワークを習得し続けられるようになる
理論を知っている範囲をフレームワークで動かす

## どのように学ぶか
TensorFLowのサンプルコードを元に、これまで扱ってきたデータセットを学習していきます。

### 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

●（例）

・重みを初期化する必要があった

・エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

●列挙：

・フォワードプロバゲーションのメソッドが必要

・交差エントロピーのメソッドが必要

・バックプロバゲーションのメソッドが必要

・推定するメソッド(predict)が必要


### 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した
「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。

In [1]:
from keras import backend as K
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

/Users/tominagashuuji/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os # ファイルの読み込み先のpath指定する時に必要
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="../../../data/Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数 目的関数を求めるのにラベルと、ロジスティック回帰のlossを出力
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法、一番良い重み・パラメーターに近づける
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

W0423 18:58:10.800657 140736299279296 deprecation.py:323] From /Users/tominagashuuji/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 0, loss : 8.1215, val_loss : 11.4562, acc : 0.750, val_acc : 0.688
Epoch 1, loss : 0.0000, val_loss : 1.7156, acc : 1.000, val_acc : 0.750
Epoch 2, loss : 0.0000, val_loss : 2.3384, acc : 1.000, val_acc : 0.875
Epoch 3, loss : 0.0000, val_loss : 0.4498, acc : 1.000, val_acc : 0.938
Epoch 4, loss : 0.0000, val_loss : 3.2183, acc : 1.000, val_acc : 0.750
Epoch 5, loss : 0.0000, val_loss : 1.9139, acc : 1.000, val_acc : 0.812
Epoch 6, loss : 0.0006, val_loss : 5.8079, acc : 1.000, val_acc : 0.750
Epoch 7, loss : 0.8854, val_loss : 11.5625, acc : 0.750, val_acc : 0.500
Epoch 8, loss : 2.4520, val_loss : 10.1613, acc : 0.750, val_acc : 0.750
Epoch 9, loss : 5.9777, val_loss : 21.2610, acc : 0.750, val_acc : 0.438
test_acc : 0.500


#### 「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。
・重みを初期化する必要があった：91行目で重みに初期値を設定している。

・エポックのループが必要だった：130行目で実装されている。Numpyと同様の実装方法。

・フォワードプロバゲーションのメソッドが必要：  
・交差エントロピーのメソッドが必要：  
・バックプロバゲーションのメソッドが必要：  
・推定するメソッド(predict)が必要：numpyで実装した時はそれぞれでメソッドを分けていたが、tensorflowでは全てを一度に実装している。  

### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに
含まれる3種類全てを分類できるモデルを作成してください。

2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

In [4]:
# データセットの読み込み
dataset_path ="../../../data/Iris.csv"
df = pd.read_csv(dataset_path)

In [5]:
# # データフレームから条件抽出＞条件無しの全件抽出する（3種類の目的変数で分類するから）
# df = df[(df["Species"b] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

In [6]:
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=='Iris-setosa'] = 2
y = y.astype(np.int)[:, np.newaxis] # dtype をintに変換している。

In [7]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [8]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [9]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [10]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

In [11]:
# ネットワーク構造の読み込み
logits = example_net(X)

# ここは２クラス分類特有の処理なので、変更する。
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# ここは２クラス分類特有の処理なので、変更する。
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

In [12]:
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init) # 初期化
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 136.1617, val_loss : -39.3823, acc : 0.333, val_acc : 0.708
Epoch 1, loss : 105.5698, val_loss : -84.1504, acc : 0.333, val_acc : 0.708
Epoch 2, loss : 5.0226, val_loss : -130.4692, acc : 0.167, val_acc : 0.583
Epoch 3, loss : 29.8799, val_loss : -170.2170, acc : 0.333, val_acc : 0.708
Epoch 4, loss : -14.9408, val_loss : -229.4171, acc : 0.167, val_acc : 0.625
Epoch 5, loss : -21.6071, val_loss : -304.5970, acc : 0.333, val_acc : 0.708
Epoch 6, loss : -32.5426, val_loss : -404.6643, acc : 0.333, val_acc : 0.667
Epoch 7, loss : -54.7993, val_loss : -545.1113, acc : 0.333, val_acc : 0.625
Epoch 8, loss : -81.1332, val_loss : -708.2380, acc : 0.333, val_acc : 0.625
Epoch 9, loss : -119.5643, val_loss : -911.8044, acc : 0.333, val_acc : 0.625
test_acc : 0.500


### 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。

In [13]:
Ames = pd.read_csv("../../../data/train.csv")
Ames.head(4)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000


In [14]:
# GrLivAreaとYearBuiltと目的変数SalePriceを抜き出す
Ames = Ames.loc[:, ["GrLivArea", "YearBuilt", "SalePrice"]]
Ames.head()

,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000


In [15]:
"""
TensorFlowで実装したニューラルネットワークを使いHousePricesデータの住宅価格を予測する
"""
K.clear_session()

# データフレームから条件抽出
X = Ames.loc[:, ["GrLivArea", "YearBuilt"]]
y = Ames["SalePrice"]
y = np.array(y)[:, np.newaxis]
X = np.array(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

print("X_test{}".format(X_test.shape))
print("X_train{}".format(X_train.shape))
print("y_train{}".format(y_train.shape))
print("X_val{}".format(X_train.shape))
print("y_val{}".format(y_val.shape))

# 特徴量データを正規化する
mean_train = X_train.mean(axis=0)
std_train = X_train.std(axis=0)
X_train = (X_train - mean_train) / std_train
mean_test = X_test.mean(axis=0)
std_test = X_test.std(axis=0)
X_test = (X_test - mean_test) / std_test
mean_val = X_val.mean(axis=0)
std_val = X_val.std(axis=0)
X_val = (X_val - mean_val) / std_val

#対数変換
y_test = np.log(y_test)
y_train = np.log(y_train)
y_val = np.log(y_val)


class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1 # ノード数

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])#tf.matmulはドット積（ブロードキャストしない）
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)


# 目的関数（サンプル分の誤差が出るので、平均を取る)# 指標値計算（MSE）
mse = tf.keras.losses.MeanSquaredError()
MSE_loss = mse(Y, logits)
loss_op = tf.reduce_mean(MSE_loss)

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果 
pred = tf.math.exp(logits)

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_mse = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, mse = sess.run([loss_op, MSE_loss], feed_dict={X: mini_batch_x, Y: mini_batch_y})

            total_loss += loss
            total_mse += mse
        total_loss /= n_samples
        total_mse /= n_samples

        val_loss, val_mse = sess.run([loss_op, MSE_loss], feed_dict={X: X_val, Y: y_val})
        
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, MSE : {:.3f}, val_MSE : {:.3f}".format(epoch, loss, val_loss, mse, val_mse))
    test_mse = sess.run(MSE_loss, feed_dict={X: X_test, Y: y_test})
    pred_1 = sess.run(pred,feed_dict={X: X_test, Y: y_test})
    print("test_MSE : {:.3f}".format(test_mse))
print("推定\n{}".format(pred_1[:10]))
print("正解値\n{}".format(np.exp(y_test[:10])))

#tf.summary.FileWriter("test", sess.graph)

W0423 19:22:19.516103 140736299279296 module_wrapper.py:139] From /Users/tominagashuuji/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:107: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0423 19:22:19.518062 140736299279296 module_wrapper.py:139] From /Users/tominagashuuji/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:111: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0423 19:22:19.521461 140736299279296 module_wrapper.py:139] From /Users/tominagashuuji/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



X_test(292, 2)
X_train(934, 2)
y_train(934, 1)
X_val(934, 2)
y_val(234, 1)
Epoch 0, loss : 2.7525, val_loss : 8.3327, MSE : 2.752, val_MSE : 8.333
Epoch 1, loss : 0.1950, val_loss : 1.6583, MSE : 0.195, val_MSE : 1.658
Epoch 2, loss : 0.1650, val_loss : 1.1400, MSE : 0.165, val_MSE : 1.140
Epoch 3, loss : 0.3168, val_loss : 1.0076, MSE : 0.317, val_MSE : 1.008
Epoch 4, loss : 0.4072, val_loss : 0.8707, MSE : 0.407, val_MSE : 0.871
Epoch 5, loss : 0.4528, val_loss : 0.8207, MSE : 0.453, val_MSE : 0.821
Epoch 6, loss : 0.4305, val_loss : 0.8205, MSE : 0.430, val_MSE : 0.820
Epoch 7, loss : 0.3149, val_loss : 0.6925, MSE : 0.315, val_MSE : 0.693
Epoch 8, loss : 0.2195, val_loss : 0.6205, MSE : 0.219, val_MSE : 0.620
Epoch 9, loss : 0.1362, val_loss : 0.5866, MSE : 0.136, val_MSE : 0.587
test_MSE : 0.462
推定
[[ 618779.06 ]
 [  99734.25 ]
 [  65849.75 ]
 [ 435309.4  ]
 [  95963.72 ]
 [  49922.344]
 [ 117618.7  ]
 [ 207362.1  ]
 [1416648.   ]
 [ 139386.78 ]]
正解値
[[200624.]
 [133000.]
 [110000

In [16]:
# tensorbord表示のためのコード
# tf.reset_default_graph()
# !tensorboard --logdir=test --port=8013

### 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。

3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。

スクラッチで実装したモデルの再現を目指してください。

In [17]:
"""
TensorFlowで実装したニューラルネットワークを使いMNISTを多値分類する
"""
K.clear_session()

# データセットの読み込み
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# データセットを確認
# print(X_train.shape) # (60000, 28, 28)
# print(X_test.shape) # (10000, 28, 28)
# print(X_train[0].dtype) # uint8
#print(X_train[0])

#平滑化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
#２２５で割る
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255 # 全て２５５で割っている
X_test /= 255

#trainをさらに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
# print(X_train.shape) # (48000, 784)
# print(X_val.shape) # (12000, 784)

# yをone-hot表現に変換
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
y_val_one_hot = enc.transform(y_val[:, np.newaxis])
# print(y_train.shape) # (60000,)
# print(y_train_one_hot.shape) # (60000, 10)
# print(y_test_one_hot.dtype) # float64
# print(y_val_one_hot.dtype) # float64


class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10 # ノード数

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数（softmaxで確率を出す＆誤差）サンプル分の誤差が出るので、平均を取る必要がある
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)#勾配効果法的な

# 推定
pred = tf.argmax(tf.nn.softmax(logits),1) # オリジナルで作った。セッションで出力

# 推定結果
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(logits), 1), tf.argmax(Y, 1))#正解だったかそうでないか

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_one_hot})
        
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    pred_1 = sess.run(pred,feed_dict={X: X_test, Y: y_test_one_hot})
    print("pred{}".format(pred_1[:20]))
    print("正解{}".format(y_test[:20]))
    print("test_acc : {:.3f}".format(test_acc))
    
    #tf.summary.FileWriter("test", sess.graph)

Epoch 0, loss : 1.4279, val_loss : 1.2675, acc : 0.800, val_acc : 0.637
Epoch 1, loss : 1.1581, val_loss : 0.9246, acc : 0.500, val_acc : 0.690
Epoch 2, loss : 0.7785, val_loss : 0.6529, acc : 0.800, val_acc : 0.818
Epoch 3, loss : 0.6824, val_loss : 0.4809, acc : 0.800, val_acc : 0.860
Epoch 4, loss : 0.8794, val_loss : 0.4010, acc : 0.700, val_acc : 0.891
Epoch 5, loss : 0.8354, val_loss : 0.3817, acc : 0.700, val_acc : 0.903
Epoch 6, loss : 0.5665, val_loss : 0.3918, acc : 0.800, val_acc : 0.907
Epoch 7, loss : 0.8795, val_loss : 0.3606, acc : 0.700, val_acc : 0.910
Epoch 8, loss : 0.7648, val_loss : 0.4001, acc : 0.800, val_acc : 0.906
Epoch 9, loss : 0.8649, val_loss : 0.4068, acc : 0.700, val_acc : 0.910
pred[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 8 4]
正解[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
test_acc : 0.911


In [18]:
# tensorbord表示のためのコード
# tf.reset_default_graph()
# !tensorboard --logdir=test --port=8014
